In [92]:
from dpd.wikipedia import get_wikipedia_table
from numpy import nan
import pandas as pd

pd.set_option('display.max_rows', 1000)

In [128]:
radio = pd.read_csv("Radio.csv")
radio.head(8)

,Location,Name,Frequency,Duplex,Offset,Tone,rToneFreq,cToneFreq,DtcsCode,DtcsPolarity,Mode,TStep,Skip,Comment,URCALL,RPT1CALL,RPT2CALL,DVCODE
0,1,6m Call,52.525,NaN,0.0,NaN,88.5,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2m Call,146.520,NaN,0.0,NaN,88.5,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,220 Call,223.500,NaN,0.0,NaN,88.5,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,70cm Cal,446.000,NaN,0.0,NaN,88.5,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
4,71,PAPA 01,446.580,-,5.0,Tone,127.3,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
5,72,PAPA 02,224.400,-,1.6,Tone,156.7,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
6,93,PAPA 23,446.700,-,5.0,Tone,127.3,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN
7,101,WIN 01,448.800,-,5.0,Tone,100.0,88.5,23,NN,FM,5.0,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
win_system = get_wikipedia_table("http://www.winsystem.org/repeaters/")
win_system["NAME"] = win_system["#"].map(lambda x: "WIN %02d" % int(x))
win_system.head()

,#,Name,ST/PR,Call,FREQ/Offset/PL,Coverage,AllStar or IRLP,Node #,NAME
0,1,Vista,CA,K6JSI,448.800* (-) 100.0,Oceanside / Vista,AllStar,1330,WIN 01
1,2,Palomar,CA,K6JSI,449.080* (-) 100.0,Palomar Mtn. - No. San Diego,AllStar,1310,WIN 02
2,3,Otay,CA,K6JSI,447.640* (-) 100.0,Mt. Otay - San Diego,AllStar,1320,WIN 03
3,4,Santiago,CA,K6JSI,448.060 (-) 100.0,Santiago Pk. - Orange / Riverside County,AllStar,1350,WIN 04
4,5,Breckenridge,CA,K6JSI,447.640* (-) 100.0,Breckenridge Mtn. - Bakersfield,AllStar,1350,WIN 05


In [175]:
location = 101
for i, row in win_system.iterrows():
    radio = radio[radio.Location != location]
    name = row["NAME"]
    frequency = row["FREQ/Offset/PL"][:7]
    if "Simplex" in row["FREQ/Offset/PL"]:
        duplex = nan
    else:
        duplex = row["FREQ/Offset/PL"].split("(")[1].split(")")[0]
    if duplex == " - ":
        duplex = "-"
    tone = "Tone"
    rToneFreq = row["FREQ/Offset/PL"][-5:]
    if rToneFreq == "0.0 X":
        rToneFreq = "100.0"
    DtcsCode = 23
    if rToneFreq == "g 072":
        tone = "DTCS"
        DtcsCode = "072"
        rToneFreq = "123.0"
    if rToneFreq == " 67.0":
        rToneFreq = "67.0"
    new_row = {
        'Location': 71,
         'Name': 'PAPA 01',
         'Frequency': 446.58,
         'Duplex': '-',
         'Offset': 5.0,
         'Tone': 'Tone',
         'rToneFreq': 127.3,
         'cToneFreq': 88.5,
         'DtcsCode': 23,
         'DtcsPolarity': 'NN',
         'Mode': 'FM',
         'TStep': 5.0,
         'Skip': nan,
         'Comment': nan,
         'URCALL': nan,
         'RPT1CALL': nan,
         'RPT2CALL': nan,
         'DVCODE': nan
    }
    new_row["Location"] = location
    new_row["Name"] = name
    new_row["Frequency"] = frequency
    new_row["Duplex"] = duplex
    new_row["Tone"] = tone
    new_row["rToneFreq"] = rToneFreq
    new_row["DtcsCode"] = DtcsCode
    radio = radio.append(new_row, ignore_index=True)
    #print(new_row)
    location += 1

In [151]:
papa_system = get_wikipedia_table("https://docs.google.com/spreadsheets/d/e/2PACX-1vTtPrC1lZiVXDDzC0sGm00icbuowp9OJx5aHCv0B2dOfVCDzmBNjrlc9DIHfl6Le_Z-ZxhNCFzTt4mi/pubhtml?gid=0&amp;single=true&amp;widget=true&amp;headers=false#")
papa_system.columns = papa_system.iloc[0]
papa_system.drop(papa_system.index[0], inplace=True)
papa_system.head()

,1,Status,NAME,Site Link,Frequency,EncodePL / DPL,Optional DecodePL / DPL,EmergencyPower,LastUpdate,
1,2,Online,PAPA 01,Oat Mountain,446.580-,127.3,100.0,Yes,2020-05-23,
2,3,Online,PAPA 02,Oat Mountain,224.400-,156.7,156.7,Yes,2020-05-23,
3,4,Online,PAPA 03,Santiago,446.760-,073 DPL,073 DPL,Yes,2020-11-16,
4,5,Online,PAPA 04,Saddle Peak,445.420-,127.3,141.3,,2020-08-12,
5,6,Online,PAPA 05,Signal Peak,449.280-,127.3,127.3,,2020-05-23,


In [173]:
location = 71
for i, row in papa_system.iterrows():
    if row["NAME"] != "PAPA 17":
        radio = radio[radio.Location != location]
        name = row["NAME"]
        frequency = row["Frequency"][:7]
        duplex = row["Frequency"][7]
        tone = "Tone"
        rToneFreq = row["EncodePL / DPL"]
        DtcsCode = 23
        if rToneFreq == "073 DPL":
            tone = "DTCS"
            DtcsCode = "073"
            rToneFreq = "123.0"
        new_row = {
            'Location': 71,
             'Name': 'PAPA 01',
             'Frequency': 446.58,
             'Duplex': '-',
             'Offset': 5.0,
             'Tone': 'Tone',
             'rToneFreq': 127.3,
             'cToneFreq': 88.5,
             'DtcsCode': 23,
             'DtcsPolarity': 'NN',
             'Mode': 'FM',
             'TStep': 5.0,
             'Skip': nan,
             'Comment': nan,
             'URCALL': nan,
             'RPT1CALL': nan,
             'RPT2CALL': nan,
             'DVCODE': nan
        }
        new_row["Location"] = location
        new_row["Name"] = name
        new_row["Frequency"] = frequency
        new_row["duplex"] = duplex
        new_row["tone"] = tone
        new_row["rToneFreq"] = rToneFreq
        new_row["DtcsCode"] = DtcsCode
        radio = radio.append(new_row, ignore_index=True)
        #print(new_row)
    location += 1
    if location == 76:
        location += 1

In [176]:
radio.to_csv("Radio.csv", index=False)